<a href="https://colab.research.google.com/github/heroza/Keel/blob/main/glass1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import myfunction as fu
import numpy as np
import pandas as pd
from typing_extensions import Counter
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [3]:
df = pd.read_csv('glass1.dat', header=None)


In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,negative
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,negative
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,negative
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,negative
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,negative


In [21]:
le = LabelEncoder()
X = df.loc[:, df.columns[0:8]].to_numpy()

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

y = df.loc[:, df.columns[-1]].to_numpy()
y = le.fit_transform(y)
# y_train = y_train.reshape(-1)
print(X.shape)
print(Counter(y))

(214, 8)
Counter({0: 138, 1: 76})


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Initialize the SVM classifier
clf = svm.SVC()

# Train the classifier on the training set
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred)

# Print the F1 score
print("F1 score:", f1)

F1 score: 0.0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Initialize the SVM classifier
clf = svm.SVC()

oversampler = RandomOverSampler()
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)


# Train the classifier on the training set
clf.fit(X_train_resampled, y_train_resampled)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

F1 score: 0.5185185185185185


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Initialize the SVM classifier
clf = svm.SVC()

oversampler = SMOTE()
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)


# Train the classifier on the training set
clf.fit(X_train_resampled, y_train_resampled)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred)

# Print the F1 score
print("F1 score:", f1)

F1 score: 0.6399999999999999


In [28]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
thresholds = [0.02, 0.05, 0.1, 0.3, 0.5]
ks = [3,5,7,9]
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for k in ks:
  for threshold in thresholds:
    scores = []
    for i in range(3):
      for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        # print(f"Fold {fold+1}")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        sia_model = fu.train_siamese_dense(X_train, y_train, X_test, y_test, num_classes=2, num_features=8, epochs_siamese=100, batch_size=16)

        # Initialize the SVM classifier
        clf = svm.SVC()

        X_train_resampled, y_train_resampled = fu.sia_smote_balance_dense(X_train, y_train, sia_model, threshold, maxdist_from_base=0.5, k = k)

        # Train the classifier on the training set
        clf.fit(X_train_resampled, y_train_resampled)

        # Predict the labels of the test set
        y_pred = clf.predict(X_test)

        # Calculate the F1 score
        score = fu.calculate_score(y_test, y_pred)
        scores.append(score)

        # Print the F1 score
        print(f"Score in i:{i}, fold:{fold}:", score)

    print(f'k={k} and threshold={threshold}')
    print('printing score')
    fu.print_score(scores)

training siamese network...
2/2 [==============================] - 0s 6ms/step
Loss = 0.2669447064399719, Train Accuracy = 0.7291666666666666 Test Accuracy = 0.625
2/2 [==============================] - 0s 5ms/step
Score in i:0, fold:0: [0.5581395348837209, 0.6297619047619047, 0.5835033765765117, 0.5777777777777778, 0.43333333333333335, 0.8666666666666667, 0.39285714285714285, 0.6297619047619047]
training siamese network...
2/2 [==============================] - 0s 5ms/step
Loss = 0.2628110349178314, Train Accuracy = 0.7125 Test Accuracy = 0.6428571428571429
2/2 [==============================] - 0s 5ms/step
Score in i:0, fold:1: [0.5348837209302325, 0.6428571428571428, 0.5345224838248488, 0.6, 0.42857142857142855, 1.0, 0.2857142857142857, 0.6428571428571428]
training siamese network...
2/2 [==============================] - 0s 8ms/step
Loss = 0.27733367681503296, Train Accuracy = 0.7708333333333334 Test Accuracy = 0.5714285714285714
2/2 [==============================] - 0s 7ms/step
S

In [29]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
thresholds = [0.02, 0.05, 0.1, 0.3, 0.5]
ks = [3,5,7,9]
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for k in ks:
  for threshold in thresholds:
    scores = []
    for i in range(3):
      for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        # print(f"Fold {fold+1}")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        sia_model = fu.train_siamese_dense(X_train, y_train, X_test, y_test, num_classes=2, num_features=8, epochs_siamese=100, batch_size=16)

        # Initialize the SVM classifier
        clf = svm.SVC()

        X_train_resampled, y_train_resampled = fu.sia_smote_then_smote_dense(X_train, y_train, sia_model, threshold, maxdist_from_base=0.5, k = k)

        # Train the classifier on the training set
        clf.fit(X_train_resampled, y_train_resampled)

        # Predict the labels of the test set
        y_pred = clf.predict(X_test)

        # Calculate the F1 score
        score = fu.calculate_score(y_test, y_pred)
        scores.append(score)

        # Print the F1 score
        print(f"Score in i:{i}, fold:{fold}:", score)

    print(f'k={k} and threshold={threshold}')
    print('printing score')
    fu.print_score(scores)

creating pairs
Counter({1.0: 120, 0.0: 120})
Counter({1.0: 28, 0.0: 28})
init base network
training siamese network...
evaluate
2/2 [==============================] - 0s 7ms/step - loss: 0.2164
predict train
8/8 [==============================] - 0s 2ms/step
compute train acc
predict val
2/2 [==============================] - 0s 5ms/step
compute val acc
Loss = 0.21643950045108795, Train Accuracy = 0.7166666666666667 Test Accuracy = 0.6785714285714286
2/2 [==============================] - 0s 5ms/step
Score in i:0, fold:0: [0.6046511627906976, 0.65, 0.6324555320336759, 0.5853658536585367, 0.46153846153846156, 0.8, 0.5, 0.65]
creating pairs
Counter({1.0: 120, 0.0: 120})
Counter({1.0: 28, 0.0: 28})
init base network
training siamese network...
evaluate
2/2 [==============================] - 0s 7ms/step - loss: 0.2661
predict train
8/8 [==============================] - 0s 2ms/step
compute train acc
predict val
2/2 [==============================] - 0s 5ms/step
compute val acc
Loss = 0.266